In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

// TODO(Johann) could these be moved to json configuration files?
val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val IMPORT_CONFIGURATION_FILE = "../../data/provision/import-configuration.json"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-12.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=a4894ef4-bb5d-4288-bc0c-a77c2b189cc6, username=Johann-12.1)

In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=8fcd2da0-ecb9-4053-9b23-f88c1e977477, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=073c9f4c-14eb-469d-a950-9181096de459, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=3b07db9b-a58f-40d9-8cf6-fd8a97d1e84b, name=ING RON, unit=Currency(value=RON))
AccountTO(id=5368dbef-8537-4a71-af24-1f4fbaa0c3ac, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=b4ac92b9-0bd5-4590-868b-e7bcf2d8eb1c, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=5e4f37df-56f1-485b-a224-2e9131217349, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=cc4f3936-62ef-4725-8238-9e04c637ff63, name=Other, unit=Currency(value=RON))
AccountTO(id=6d7e64df-a1dc-484c-b3f4-9840dbad19e4, name=BT RON, unit=Currency(value=RON))
AccountTO(id=4b303c2a-3952-42d6-9e68-3d4bbdd76f70, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=c8fab236-25f1-43d0-a8c8-0f5331821c68, name=Expenses)
FundTO(id=13ad7a17-cb5e-42c3-b2a7-12ba28f7d355, name=Work Income)
FundTO(id=677aa5fb-aacb-4947-9b74-da6ff5b8cba2, name=Savings)
FundTO(id=f8eb2bc0-ae22-4351-9530-f549d6cb5988, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawJson: String = File(IMPORT_CONFIGURATION_FILE).readText()
val jsonFormat = Json { ignoreUnknownKeys = true }
val importConfiguration = jsonFormat.decodeFromString<ImportConfigurationTO>(dataConfigurationRawJson)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles = File("../../data/wallet/2019")
    .listFiles()?.toList() ?: error("no files found")
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=091c4e0a-0112-4f28-8380-93fdacc0f9e9, status=COMPLETED, reason=null)